In [ ]:
# Welcome to version 3 of Jons software simulation for the readout of Mona Jarrahis room temperature plasmonic photomixer
# Hopefully this will be the final version
# Change log from version 2:
#    Solving for normalization factor of FFT to solve for absolute values
#    Much more plotting throughout simulation for the sake of others looking in
#    Comparisons of plots at different stages for lock-in and simple hoh_spec
#    Accumulation before taking intensity of demodulated time-streams
# What needs to be solved in version 3:
#    Getting a believable result for allan variance comparison
#    Figure out why demodulation causes amplitude to shoot up so high

In [ ]:
#***********************************#
#                                   #
#             Packages              #        
#                                   #
#***********************************#
import math
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import allantools as AT
from scipy import signal as sig
from scipy import stats as stat
import line_profiler
import ipywidgets as wdg
import dsp_py as dsp

In [ ]:
#***********************************#
#                                   #
#             Constants             #        
#                                   #
#***********************************#

adc_clk = 512 * 10 ** 6 # adc clock rate
#adc_clk = 512 * 10 ** 3 # adc clock rate

T_adc = 1/(adc_clk) # adc period

fpga_clk = 256 * 10 ** 6 # fpga fabric clock rate
#fpga_clk = 256 * 10 ** 3 # fpga fabric clock rate


FFT_length = 1024 # Length of FFT used in spectrometer
#FFT_length = 128 # Length of FFT used in spectrometer

accum_len = 2 ** 23 # number of clock cycles of accumulation in every dump of data
#accum_len = 2 ** 13 # number of clock cycles of accumulation in every dump of data

accum_time = ((adc_clk)/FFT_length)/accum_len # amount of time one accumulation cycle integrates (seconds)



# Put parameters for testing here

source_freq = 100 * 10 ** 6 
#source_freq = 100 * 10 ** 3
square_freq = 100 * 10 ** 3
#square_freq = 80

In [ ]:
#***********************************#
#                                   #
#            Functions              #        
#                                   #
#***********************************#


################################################
# Wave generation function:                    #
# Input: amp, frequency and amount of time     #
# Output : I and Q arrays for wave             #
################################################

def real_wave(amp, freq, time, phase=0):
    omega = 2 * np.pi * freq    
    wave = amp*np.cos(omega * time + phase)
    return wave

def cool_wave(amp, freq, time):
    omega = 2 * np.pi * freq    
    wave = amp * np.exp(1j * omega * time)
    i = np.real(wave)
    q = np.imag(wave)
    return i,q
    
#########################################################
#    Complex multipler function:                        #
#    Input: I_1, Q_1, I_2, Q_2                          #
#    Output: Mixed wave in quadrature (I_mix, Q_mix)    #
#########################################################

def c_mult(I_1, Q_1, I_2, Q_2): 
    I_mix = I_1 * I_2 - Q_1 * Q_2 
    Q_mix = I_1 * Q_2 + Q_1 * I_2 
    return I_mix, Q_mix

def real_mix(wave_1, wave_2):
    mix_wave = wave_1 * wave_2
    return mix_wave

#########################################################
#    Other Functions:                                   #
#    FFT for I,Q signal                                 #
#    Noise Inclusion                                    # 
#    Chopping                                           #
#    File Saving                                        #
#    Allan Variance                                     # 
#########################################################

def fft(signal):
    spectrum = np.fft.rfft(signal, n = FFT_length)
    intensity = np.real(spectrum)**2 + np.imag(spectrum)**2
    return spectrum, intensity

def fft_IQ(signal_i, signal_q):
    spectrum = np.fft.fft(signal_i + 1j*(signal_q), n = FFT_length)
    intensity = np.real(spectrum)**2 + np.imag(spectrum)**2
    return spectrum, intensity

def noisify(signal_i, signal_q, POWER):
    w_noise = np.random.normal(0, POWER, signal_i.shape)
    noisy_sig_i = signal_i * w_noise
    w_noise = np.random.normal(0, POWER, signal.shape)
    noisy_sig_q = signal_q * w_noise
    return noisy_sig_i, noisy_sig_q

def GET_TO_DA_CHOPPAH(signal, timespace):
    sq_wave = 0.5 * (sig.square(2 * np.pi * square_freq * timespace) + 1)
    chopped_wave = signal * sq_wave
    return chopped_wave

def complex_choppin(sig_i, sig_q, clk_chop, timespace):
    sq_wave_i = 0.5 * (sig.square(2 * np.pi * clk_chop * timespace) + 1) 
    sq_wave_q = 0.5 * (sig.square(2 * np.pi * clk_chop * timespace + (np.pi/4)) + 1)
    chop_sig_i, chop_sig_q = c_mult(sig_i, sig_q, sq_wave_i, sq_wave_q)
    return chop_sig_i, chop_sig_q
    
def save_data(file_name, data):
    file = open(file_name, 'w')
    writer = csv.writer(file)
    writer.writerow(data)
    
def allan_var(timestream, res=30):
    rate = 1/(accum_time)
    tau = np.logspace(0, run_time/5, res)
    (tau2, adevs, adev_err, n) = AT.oadev(timestream, rate, data_type="freq", taus=tau)
    avars = np.square(adevs)
    white_line = (avars[0]*(tau2**-1))
    return avars, white_line, adev_err, tau2    

def magify(sig): #takes the magnitude of a complex value/vector
    mag = np.sqrt(np.square(np.real(sig)) + np.square(np.imag(sig)))
    return mag

In [ ]:
################################################
# Allan Variance Plotter:                      #
# Input: Timestream of data and bin number     #
# Output : Plot of allan variance              #
################################################


def allan_plot(data1, chan):
    num_sec = len(data1)/(1/accum_time)
    #tau = np.logspace(0, 1, 30)
    tau = np.logspace(0, np.log10(num_sec/5), 30)
    rate = 1/accum_time # 1/16 seconds of integration per sample
    
    # now take data and push through allan deviation function 
    (tau2, adevs, adev_err, n) = AT.oadev(data1, rate, data_type="freq", taus=tau)
    avars = np.square(adevs) # square allan dev to get allan var
    # Make white noise t^-1 line
    white_line = (avars[0]*(tau2**-1))  
    
    
    # Plot ur shit bro                   
    plot = plt.loglog(tau2, avars) 
    plt.loglog(tau2,white_line)   
   # plt.errorbar(tau2, avars, yerr = (avars[::]/np.sqrt((num_sec/tau2[::]))), ecolor='g')
    plt.title('Allan Variance for Lock-in Spectrometer (Bin %d)'%(chan))
    plt.xlabel('Integration Times (s)')
    plt.ylabel('Power^2 (arbitrary(?) units)')
    plt.show()

In [ ]:
################################################
# Allan Variance Comparitor:                   #
# Input: 2 Timestreams of Data and Bin Number  #
# Output : Plot comparing allan variances      #
################################################


def allan_plot_compare(data1, data2, chan):
    
    # First, figure out how long the timestreams are (assuming a 23 bit accumulator)
    num_sec = len(data1)/(1/accum_time)
    #tau = np.logspace(0, 1, 30)
    tau = np.logspace(0, np.log10(num_sec/5), 30)
    rate = 16 # around 1/16 seconds of integration per sample
    
    # Now take data and push through allan deviation function 
    (tau2, adevs, adev_err, n) = AT.oadev(data1, rate, data_type="freq", taus=tau)
    # Square allan dev to get allan var
    avars = np.square(adevs)
    # Make white noise t^-1 line
    white_line = (avars[0]*(tau2**-1))  
    
    # now for second set of data
    (tau3, adevs2, adev_err2, n2) = AT.oadev(data2, rate, data_type="freq", taus=tau)
    avars2 = np.square(adevs2) # square allan dev to get allan var
    white_line2 = (avars2[0]*(tau2**-1)) 
    
    # Plot ur shit bro
    fig, axs = plt.subplots(2, sharex=True)
    axs[0].loglog(tau2, avars, label ="hoh_spec allan variance") 
    axs[0].loglog(tau2, avars2, label ="lock-in allan variance")
    axs[0].loglog(tau2,white_line)  
    axs[0].loglog(tau2,white_line2)
    axs[0].errorbar(tau2, avars, yerr = 2*(avars[::]/np.sqrt((num_sec/tau2[::]))), ecolor='g')
    axs[0].errorbar(tau2, avars2, yerr = 2*(avars2[::]/np.sqrt((num_sec/tau2[::]))), ecolor='g')
   
    ratio = avars/avars2
    axs[1].loglog(tau2, ratio)
    
    plt.title('Allan Variance Comparison for Lock-in Spectrometer (Bin %d)'%(chan))
    plt.xlabel('Integration Times (s)')
    plt.ylabel('Power^2 (arbitrary(?) units)')
    plt.show()
    
    return(np.average(ratio))
    

In [ ]:
###############################################################################################
#                                                                                             #
#                                  Begin Main Simulation Script                               #
#                                                                                             #
###############################################################################################

In [ ]:
#***********************************#
#                                   #
#           Make FFT Frames         #        
#                                   #
#***********************************#

    ##### for a 1024 pt FFT, one FFT frame will take 1024/adc_clk (seconds) ##### 


    # frame time == Time for 1 FFT to populate
frame_time = FFT_length / (adc_clk)

    # frame_freq == how many FFT frames are created in one second
frame_freq = adc_clk/FFT_length

    #The number of fft frames to be created equals the run time of the test over the amount of time it takes to FFT a frame                        

    # accum_frames == number of FFT frames created over one accumulation
accum_frames = int(accum_time/frame_time)

    # Now create array which contains frequency span of FFT in order to have correctly scaled FFT plots
timestep = 1/adc_clk
    
    # note this value comes from the linspace used for the time array. length/number of samples
    # this array will be used to properly map the x-axis in Fourier space plots
fft_freq = np.fft.fftfreq(FFT_length, d=timestep)
rfft_freq = np.fft.rfftfreq(FFT_length, d=timestep)


In [ ]:
###############################################################################################
#                                                                                             #
#                              Hoh_spec simulation (no lock-in) (White Noise)                 #
#                                                                                             #
###############################################################################################

In [ ]:
#***********************************#
############################        #
# User Inputs:             #        #
#    Channel and Run time  #        #
############################        #
chan = 100                          #
run_time = 100                     #
#***********************************#
    
    # Define how many seconds of data should be simulated and how many accumulations are required

num_accum = int(run_time * (1/accum_time)) # number of total accumulations in simulation time

    # create file name for saving data in csv if desired

file_name = '%d_sec_hohspec_sim_chan_%d.csv'%(run_time, chan)    

    #Make an empty array to be filled with accumulations

spec_accums_i = np.zeros(num_accum)
spec_accums_q = np.zeros(num_accum)


#***********************************#
#                                   #
#           Creating Time           #        
#                                   #
#***********************************#

for i in range(num_accum):
    print('we are on accumulation number %d out of %d'%(i, num_accum))

        ##### Create an array with the times of the FFT frames #####

    frame_times = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames)  )

        ############################################################################################################    
        # Create an array of times that will be used to create the "pieces" of the wave                            #  
        # Populate time array with lengths to be used later                                                        #
        # This is an absolutely crazy vectorization of a previous loop I had, but it runs 100 times faster. Sorry. #
        ############################################################################################################

    timespace = np.linspace(np.linspace(frame_times[0], frame_times[1], FFT_length), 
                            np.linspace(frame_times[accum_frames-2], frame_times[accum_frames-1], FFT_length),
                            num = accum_frames-2)


#***********************************#
#                                   #
#            Signals                #
#   (Creation and Timestreaming)    #
#                                   #
#***********************************#         

    # create white nose data set
    signal =np.random.normal(0, 0.1, size=np.shape(timespace)) 
    

    # take spectrum of signal 
    spectra = np.fft.rfft(signal, n = FFT_length)   
   
    #plt.plot(fft_freq, np.real(spectra[4])**2 + np.imag(spectra[4])**2)
    #plt.title('Simple Spec timestream intensity')
    #plt.show()
        
        # To get timestreams, convert from rows of freq and columns of time to rows of time streams and columns of freq
        # I'm pretty proud of how cheeky this trick is tbh, all you need is a transpose, no array slicing and dicing

    t_streams = np.transpose(spectra)
    t_streams_i = np.real(t_streams)
    t_streams_q = np.imag(t_streams)
    #plt.plot(t_streams_i[4])
    #plt.show()

     
    # Find intensity of the timestream equivalent to firmware design
    intensity = np.square(t_streams_i) + np.square(t_streams_q)
    magnitude = np.sqrt(intensity)
    #plt.plot(intensity[100])
    #plt.title('Simple Spec Accumulator Timestream')
    #plt.show()
    
    spec_accums_i[i] = np.sum(t_streams_i)
    spec_accums_q[i] = np.sum(t_streams_q)

#print(np.average(spec_accums))



In [ ]:
###############################################################################################
#                                                                                             #
#                              Hoh_spec simulation (no lock-in) (100 Mhz tone)                #
#                                                                                             #
###############################################################################################

In [ ]:
#***********************************#
############################        #
# User Inputs:             #        #
#    Channel and Run time  #        #
############################        #
chan = 100                          #
run_time = 0.25                      #
#***********************************#
    
    # Define how many seconds of data should be simulated and how many accumulations are required

num_accum = int(run_time * (1/accum_time)) # number of total accumulations in simulation time

    # create file name for saving data in csv if desired

file_name = '%d_sec_hohspec_sim_chan_%d.csv'%(run_time, chan)    

    #Make an empty array to be filled with accumulations

spec_accums_i = np.zeros(num_accum)
spec_accums_q = np.zeros(num_accum)


#***********************************#
#                                   #
#           Creating Time           #        
#                                   #
#***********************************#

for i in range(num_accum):
    print('we are on accumulation number %d out of %d'%(i, num_accum))

        ##### Create an array with the times of the FFT frames #####

    frame_times = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames)  )

        ############################################################################################################    
        # Create an array of times that will be used to create the "pieces" of the wave                            #  
        # Populate time array with lengths to be used later                                                        #
        # This is an absolutely crazy vectorization of a previous loop I had, but it runs 100 times faster. Sorry. #
        ############################################################################################################

    timespace = np.linspace(np.linspace(frame_times[0], frame_times[1], FFT_length), 
                            np.linspace(frame_times[accum_frames-2], frame_times[accum_frames-1], FFT_length),
                            num = accum_frames-2)


#***********************************#
#                                   #
#            Signals                #
#   (Creation and Timestreaming)    #
#                                   #
#***********************************#         

    # tone of interest
    signal = real_wave(1, source_freq, timespace)

    dt = timespace[0][11] - timespace[0][10]

    # take spectrum of signal 
    spectra = (np.fft.fft(signal, n = FFT_length))*(2/FFT_length) 
    #spectra = (np.fft.rfft(signal, n = FFT_length))*(dt*1/frame_time) 
    spec_mag = np.sqrt(np.square(np.real(spectra)) + np.square(np.imag(spectra)))

    # Plot figures to solve for normalization
    #fig = plt.figure()
    #plt.subplot(2,2,1)
    #plt.plot(signal[100])
    
    #plt.subplot(2,2,2)
    #plt.plot(fft_freq, spec_mag[100])
    #plt.show()
    #damax = (np.argmax(spec_mag[100]))
    #print('Spectra has a maximum magnitude of %d'%(damax))
    #print(spec_mag[100][damax])
    #plt.plot(fft_freq, np.real(spectra[4])**2 + np.imag(spectra[4])**2)
    #plt.title('Simple Spec timestream intensity')
    #plt.show()
        
        # To get timestreams, convert from rows of freq and columns of time to rows of time streams and columns of freq
        # I'm pretty proud of how cheeky this trick is tbh, all you need is a transpose, no array slicing and dicing

    t_streams = np.transpose(spectra)
    t_streams_i = np.real(t_streams)
    t_streams_q = np.imag(t_streams)
    #plt.plot(t_streams_i[4])
    #plt.show()

     
    # Find intensity of the timestream equivalent to firmware design
    intensity = np.square(t_streams_i) + np.square(t_streams_q)
    #plt.plot(intensity[100])
    #plt.title('Simple Spec Accumulator Timestream')
    #plt.show()

    spec_accums_i[i] = np.sum(t_streams_i)
    spec_accums_q[i] = np.sum(t_streams_q)

#print(np.average(spec_accums))



In [ ]:
###############################################################################################
#                                                                                             #
#                                        Lock-in Simulation (White Noise)                     #
#                                                                                             #
###############################################################################################

In [ ]:
#***********************************#
############################        #
# User Inputs:             #        #
#    Channel and Run time  #        #
############################        #
chan = 200                          #
run_time = 1000                     #
#***********************************#
        
    # Define how many seconds of data should be simulated and how many accumulations are required

num_accum = int(run_time * (1/accum_time)) # number of total accumulations in simulation time

    # create file name for saving data in csv if desired

file_name = '%d_sec_lockin_sim_chan_%d.csv'%(run_time, chan)    

    #Make an empty array to be filled with accumulations

lock_accums_i = np.zeros(num_accum)
lock_accums_q = np.zeros(num_accum)

#***********************************#
#                                   #
#           Creating Time           #        
#                                   #
#***********************************#

for i in range(num_accum):
    print('we are on accumulation number %d out of %d'%(i+1, num_accum))

        ##### Create an array with the times of the FFT frames #####

    frame_times = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames)  )

        ############################################################################################################    
        # Create an array of times that will be used to create the "pieces" of the wave                            #  
        # Populate time array with lengths to be used later                                                        #
        # This is an absolutely crazy vectorization of a previous loop I had, but it runs 100 times faster. Sorry. #
        ############################################################################################################

    timespace = np.linspace(np.linspace(frame_times[0], frame_times[1], FFT_length), 
                            np.linspace(frame_times[accum_frames-2], frame_times[accum_frames-1], FFT_length),
                            num = accum_frames-2)


#***********************************#
#                                   #
#            Signals                #
#   (Creation and Timestreaming)    #
#                                   #
#***********************************#

    signal =np.random.normal(0, 0.1, size=np.shape(timespace))

        # Chop up white noise at chop frequency (comment next line to get unchopped results)

    sq_wave = 0.5 * (sig.square(2 * np.pi * square_freq * timespace) + 1)
    choppa_signal = signal * sq_wave

        
        # Now put the choppeed noise signal through PFB

    spectra = np.fft.fft(choppa_signal, n = FFT_length)
    

        # Once again, take transpose of FFT matrix to get channel timestreams

    t_streams = np.transpose(spectra)
    t_streams_i = np.real(t_streams)
    t_streams_q = np.imag(t_streams)
    
    intensity = np.square(t_streams_i) + np.square(t_streams_q)
    #plt.plot(intensity[100])
    #plt.title('timestream intensity')
    #plt.xlim(0,50)
    #plt.show()
    #print(np.sum(intensity[100])) 
    #########################################
    #    Mixing Channel Timestreams Down    #
    #########################################   

        # Create time array to control internally generated wave 

    timespace2 = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames)-2)

        # Create generated signal inside FPGA at square wave frequency 

    #chop_i, chop_q = cool_wave(1, square_freq, timespace2)
    chop_i = 0.5 * (sig.square(2 * np.pi * square_freq * timespace2) + 1) 
    chop_q = 0.5 * (sig.square(2 * np.pi * square_freq * timespace2 + (np.pi/2)) + 1) 
    #chop_i, chop_q = np.array(list(chop_i).pop()), np.array(list(chop_q).pop())
            
        # Mix together timestreams and chops

    
    downmix_i, downmix_q = c_mult(t_streams_i[chan], t_streams_q[chan], chop_i, chop_q)
    

    downmix_mag = np.square(downmix_i) + np.square(downmix_q)

    #if i == int((num_accum/2)):
    #plt.plot(downmix_mag)
    #plt.title('Demodulated Signal Intensity')
    #plt.xlim(0, 50)
    #plt.show()
    
    # just plotting things for fun here, comment out later
    #fig, axs = plt.subplots(2, sharex=False)
    #axs[0].plot(t_streams_i[chan])
    #axs[1].plot(downmix_mag)
    #plt.xlim(0,1000)
    #plt.show()

        # Now integrate the timestream as is done in firmware

    accum_i = np.sum(downmix_i)
    accum_q = np.sum(downmix_q)
    #print(accum)

        # Plop the accumulation data into our accumulation array for further faffing about

    lock_accums_i[i] = accum_i
    lock_accums_q[i] = accum_q
    
print(np.average(lock_accums))

In [ ]:
###############################################################################################
#                                                                                             #
#                                        Lock-in Simulation (100 Mhz Tone)                    #
#                                                                                             #
###############################################################################################

In [ ]:
#***********************************#
############################        #
# User Inputs:             #        #
#    Channel and Run time  #        #
############################        #
chan = 100                          #
run_time = 0.25                      #
#***********************************#
        
    # Define how many seconds of data should be simulated and how many accumulations are required

num_accum = int(run_time * (1/accum_time)) # number of total accumulations in simulation time

    # create file name for saving data in csv if desired

file_name = '%d_sec_lockin_sim_chan_%d.csv'%(run_time, chan)    

    #Make an empty array to be filled with accumulations

lock_accums_i = np.zeros(num_accum)
lock_accums_q = np.zeros(num_accum)

sig_max = 0

#***********************************#
#                                   #
#           Creating Time           #        
#                                   #
#***********************************#

for i in range(num_accum):
    print('we are on accumulation number %d out of %d'%(i+1, num_accum))

        ##### Create an array with the times of the FFT frames #####

    frame_times = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames)  )

        ############################################################################################################    
        # Create an array of times that will be used to create the "pieces" of the wave                            #  
        # Populate time array with lengths to be used later                                                        #
        # This is an absolutely crazy vectorization of a previous loop I had, but it runs 100 times faster. Sorry. #
        ############################################################################################################

    timespace = np.linspace(np.linspace(frame_times[0], frame_times[1], FFT_length), 
                            np.linspace(frame_times[accum_frames-2], frame_times[accum_frames-1], FFT_length),
                            num = accum_frames-2)


#***********************************#
#                                   #
#            Signals                #
#   (Creation and Timestreaming)    #
#                                   #
#***********************************#

    # tone of interest
    signal = real_wave(1, source_freq, timespace)

       
        # Chop up signal at chop frequency (comment next line to get unchopped results)

    sq_wave = 0.5 * (sig.square(2 * np.pi * square_freq * timespace) + 1)
    choppa_signal = signal * sq_wave

        
        # Now put the choppeed noise signal through PFB

    spectra = (np.fft.fft(choppa_signal, n = FFT_length))*(2/FFT_length) 
    #spectra = spectra*(2/FFT_length)
    if i == 0:
        #sig_max = np.argmax(sig_intensity)
        #chan = sig_max
        # Plot figures to solve for normalization
        spec_mag = np.sqrt(np.square(np.real(spectra)) + np.square(np.imag(spectra)))
        
        fig = plt.figure()
        plt.subplot(2,2,1)
        plt.plot(choppa_signal[100])

        plt.subplot(2,2,2)
        plt.plot(fft_freq, spec_mag[100])
        plt.show()

        #print('Spectra has a maximum magnitude of %d'%(np.max(spec_mag[100])))
        
        # Once again, take transpose of FFT matrix to get channel timestreams
    print(spectra[100])
    t_streams = np.transpose(spectra)
    t_streams_i = np.real(t_streams)
    t_streams_q = np.imag(t_streams)
    
    intensity = np.square(t_streams_i) + np.square(t_streams_q)
    #plt.plot(intensity[100])
    #plt.title('timestream intensity')
    #plt.xlim(0,50)
    #plt.show()
    #print(np.sum(intensity[100])) 
    #########################################
    #    Mixing Channel Timestreams Down    #
    #########################################   

        # Create time array to control internally generated wave 

    timespace2 = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames)-2)

        # Create generated signal inside FPGA at square wave frequency 

    chop_i, chop_q = cool_wave(1, square_freq, timespace2)
    #chop_i = 0.5 * (sig.square(2 * np.pi * square_freq * timespace2) + 1) 
    #chop_q = 0.5 * (sig.square(2 * np.pi * square_freq * timespace2 + (np.pi/2)) + 1) 
    #chop_i, chop_q = np.array(list(chop_i).pop()), np.array(list(chop_q).pop())
            
        # Mix together timestreams and chops

    
    downmix_i, downmix_q = c_mult(t_streams_i[200], t_streams_q[200], chop_i, chop_q)
    #downmix_i, downmix_q = c_mult(t_streams_i[200], 1, chop_i, chop_q)

    downmix_mag = np.square(downmix_i) + np.square(downmix_q)

    #if i == int((num_accum/2)):
    #plt.plot(downmix_mag)
    #plt.title('Demodulated Signal Intensity')
    #plt.xlim(0, 50)
    #plt.show()
    
    # just plotting things for fun here, comment out later
    #fig, axs = plt.subplots(2, sharex=False)
    #axs[0].plot(t_streams_i[chan])
    #axs[1].plot(downmix_mag)
    #plt.xlim(0,1000)
    #plt.show()

        # Now integrate the timestream as is done in firmware

    accum_i = np.sum(downmix_i)
    accum_q = np.sum(downmix_q)
    #print(accum)

        # Plop the accumulation data into our accumulation array for further faffing about

    lock_accums_i[i] = accum_i
    lock_accums_q[i] = accum_q
    
#print(np.average(lock_accums))

In [ ]:
###############################################################################################
#                                                                                             #
#                                Making Fun Plot of Fun Things                                #
#                                                                                             #
###############################################################################################

In [ ]:
#Plottiing Chop Signal
plt.plot(chop_i)
#plt.plot(chop_q)
plt.xlim(17500,17600)

In [ ]:
#plt.plot(downmix_i)
#plt.plot(downmix_q)
plt.plot(downmix_mag)
#plt.xlim(18500,18700)

In [ ]:
fig = plt.figure()

wtf_fft = np.fft.fft(downmix_mag, n = FFT_length)
wtf_mag = magify(wtf_fft)
plt.plot(wtf_mag)

In [ ]:
fig = plt.figure()
plt.subplot(2,2,1)
plt.plot(downmix_i)
plt.xlim(18500,18800)
plt.subplot(2,2,2)
plt.plot(downmix_q)
plt.xlim(18500,18800)
plt.subplot(2,2,3)
plt.plot(downmix_mag)
plt.xlim(18500,18800)

In [ ]:
plt.plot(t_streams_i[200])

plt.xlim(17500,19000)

In [ ]:
plt.plot(t_streams_q[200])

plt.xlim(17500,17700)

In [ ]:
plt.plot(chop_i)
plt.xlim(17500,17700)

In [ ]:
plt.plot(downmix_mag)

In [ ]:
###############################################################################################
#                                                                                             #
#                                        Ultra-Simple Testing Environment                     #
#                                                                                             #
###############################################################################################

In [ ]:
# Solving for the normalization factor of the FFT
testspace = np.linspace(0,32,1024) #32 Hz sampling speed
teststep = 32/1024
test_fft_freq = np.fft.rfftfreq(FFT_length, d=teststep)
    
test_freq = 32/5.12
test_signal = real_wave(1, test_freq, testspace)
fig = plt.figure()
plt.subplot(2,2,1)
plt.plot(testspace, test_signal)
plt.xlim(0,3)

test_spec = np.abs(np.fft.rfft(test_signal, n = FFT_length)) * (2/FFT_length)
plt.subplot(2,2,2)
plt.plot(test_fft_freq, test_spec)
#plt.xlim(0,10)
damax = (np.argmax(test_spec))
print('Spectra has a maximum amplitude of '+ str(test_spec[damax]) + ' at ' + str(damax/32) + ' MHz')



In [ ]:
test_i, test_q = cool_wave(1, 2, testspace)
test_chop_i, test_chop_q = complex_choppin(test_i, test_q, 0.1, testspace)
plt.plot(test_chop_i)

In [ ]:
test_i, test_q = cool_wave(1, 2, testspace)
test_mix_i, test_mix_q = cool_wave(1, 2, testspace)
out_i, out_q = c_mult(test_i, test_q, test_mix_i, test_mix_q)
out_mag = np.sqrt(np.square(out_i) + np.square(out_q))
plt.plot(out_mag)

In [ ]:
# Lets see the relationship between f_source and f_samp
# make a linear space with a sampling rate of 64 Hz
testspace = np.linspace(0,16,1024) #64 Hz sampling speed
teststep = 16/1024
test_fft_freq = np.fft.fftfreq(FFT_length, d=teststep)
test_rfft_freq = np.fft.rfftfreq(FFT_length, d=teststep)

# make a linear space of frequencies from 0.5 Hz to 32 Hz with 0.5 Hz step
test_tones = np.linspace(0.5, 8, 64)
amplitudes = []

# now iterate through test tones and see what normalized DFT amps they output
for i in test_tones:
    test_signal = real_wave(1, i, testspace)
    test_spec = np.abs(np.fft.fft(test_signal, n = FFT_length)) * (2/FFT_length)
    damax = (np.argmax(test_spec))
    peak_amp = test_spec[damax]
    #print(peak_amp)
    amplitudes.append(peak_amp)

plt.plot(amplitudes[0:63])

In [ ]:
test_tones[30]

In [ ]:
k = []
norm_amp = []
test_tones = np.linspace(0.5, 8, 64)
for i in test_tones:
    val = (i*FFT_length)/64
    #print(val)
    k.append(val)
i=0
while i < len(test_tones):
    test_signal = real_wave(1, test_tones[i], testspace)
    test_spec = np.abs(np.fft.rfft(test_signal, n = FFT_length))
    #plt.plot(test_spec)
    #plt.show()
    damax = (np.argmax(test_spec))
    #print(damax, k[i])
    peak_amp = (test_spec[damax]*2*(1-np.exp((-1j*2*np.pi*(damax-k[i]))/FFT_length)))/(1-np.exp(-1j*2*np.pi*(damax-k[i])))
    final_peak = np.sqrt(np.square(np.real(peak_amp)) + np.square(np.imag(peak_amp)))
    print(test_tones[i], final_peak)
    norm_amp.append(final_peak)
    i+=1
